In [1]:
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-chinese"
model_path = "/Users/huangxinzhe/code/huggingface_note/hugging_face_transformers/models/bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

/Users/huangxinzhe/opt/anaconda3/envs/llm9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 第一步：分词
sequence = "美国的首都是华盛顿特区"
tokens = tokenizer.tokenize(sequence)
print(tokens)

['美', '国', '的', '首', '都', '是', '华', '盛', '顿', '特', '区']


In [4]:
# 第二步：映射
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277]


In [5]:
# 一步完成分词与映射
token_ids_e2e = tokenizer.encode(sequence)
print(token_ids_e2e)

[101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102]


In [6]:
tokenizer.decode(token_ids)

'美 国 的 首 都 是 华 盛 顿 特 区'

In [7]:
tokenizer.decode(token_ids_e2e)

'[CLS] 美 国 的 首 都 是 华 盛 顿 特 区 [SEP]'

In [8]:
sequence_batch = ["美国的首都是华盛顿特区", "中国的首都是北京"]
embedding_batch = tokenizer(sequence_batch)
print(embedding_batch)

{'input_ids': [[101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102], [101, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [9]:
# 优化下输出结构
for key, value in embedding_batch.items():
    print(f"{key}: {value}\n")

input_ids: [[101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102], [101, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102]]

token_type_ids: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]



### 添加新 Token

当出现了词表或嵌入空间中不存在的新Token，需要使用 Tokenizer 将其添加到词表中。 Transformers 库提供了两种不同方法：

- add_tokens: 添加常规的正文文本 Token，以追加（append）的方式添加到词表末尾。
- add_special_tokens: 添加特殊用途的 Token，优先在已有特殊词表中选择（`bos_token, eos_token, unk_token, sep_token, pad_token, cls_token, mask_token`）。如果预定义均不满足，则都添加到`additional_special_tokens`。

#### 添加常规 Token

先查看已有词表，确保新添加的 Token 不在词表中：

In [10]:
len(tokenizer.vocab.keys())

21128

In [11]:
from itertools import islice

# 使用 islice 查看词表部分内容
for key, value in islice(tokenizer.vocab.items(), 10):
    print(f"{key}: {value}")

276: 11173
##安: 15185
##语: 19484
螯: 6088
##戡: 15834
hall: 11049
##诲: 19488
fc2: 11362
##邳: 19995
齡: 7972


In [12]:
new_tokens = ["天干", "地支"]

In [13]:
# 将集合作差结果添加到词表中
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())

In [14]:
new_tokens

{'地支', '天干'}

In [15]:
tokenizer.add_tokens(list(new_tokens))

2

In [16]:
# 新增加了2个Token，词表总数由 21128 增加到 21130
len(tokenizer.vocab.keys())

21130

### 添加特殊Token（审慎操作）

In [17]:
new_special_token = {"sep_token": "NEW_SPECIAL_TOKEN"}

In [18]:
tokenizer.add_special_tokens(new_special_token)

1

In [19]:
# 新增加了1个特殊Token，词表总数由 21128 增加到 21131
len(tokenizer.vocab.keys())

21131

### 使用 `save_pretrained` 方法保存指定 Model 和 Tokenizer 

借助 `AutoClass` 的设计理念，保存 Model 和 Tokenizer 的方法也相当高效便捷。

假设我们对`bert-base-chinese`模型以及对应的 `tokenizer` 做了修改，并更名为`new-bert-base-chinese`，方法如下：

```python
tokenizer.save_pretrained("./models/new-bert-base-chinese")
model.save_pretrained("./models/new-bert-base-chinese")
```

保存 Tokenizer 会在指定路径下创建以下文件：
- tokenizer.json: Tokenizer 元数据文件；
- special_tokens_map.json: 特殊字符映射关系配置文件；
- tokenizer_config.json: Tokenizer 基础配置文件，存储构建 Tokenizer 需要的参数；
- vocab.txt: 词表文件；
- added_tokens.json: 单独存放新增 Tokens 的配置文件。

保存 Model 会在指定路径下创建以下文件：
- config.json：模型配置文件，存储模型结构参数，例如 Transformer 层数、特征空间维度等；
- pytorch_model.bin：又称为 state dictionary，存储模型的权重。

In [ ]:
tokenizer.save_pretrained("./models/new-bert-base-chinese")

In [ ]:
model.save_pretrained("./models/new-bert-base-chinese")